Funciones generales de las capas (Clase base) --- 0:00 min
===

* Última modificación: Marzo 7, 2022 | YouTube

* Adaptado de: https://keras.io/api/layers/base_layer/

Importación de librerías
---

In [1]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf

tf.__version__

'2.8.0'

Clase base Layer
---

Esta es la clase base y cubre todas las funciones que son generales a las capas de procesamiento en un modelo de redes neuronales.

```python
tf.keras.layers.Layer(
    # -------------------------------------------------------------------------
    # ?si es True los pesos son modificables durante el entrenamiento.
    trainable=True, 
    # -------------------------------------------------------------------------
    # string con el nombre de la capa
    name=None, 
    # -------------------------------------------------------------------------
    # tipo de dato para los célculos
    dtype=None, 
)
```

In [2]:
#
# Ejemplo 1.
# Capa que implementa el modelo y = w x + b
#
class SimpleDense(tf.keras.layers.Layer):
    #
    def __init__(
        self,
        units=32,
    ):
        super(SimpleDense, self).__init__()
        self.units = units

    def build(self, input_shape):
        #
        # Inicializa un generador de aleatorios
        #
        w_init = tf.random_normal_initializer()

        #
        # Crea la variable `w` con un valor inicial aleatorio
        #
        self.w = tf.Variable(
            initial_value=w_init(
                shape=(input_shape[-1], self.units),
                dtype="float32",
            ),
            trainable=True,
        )

        #
        # Inicializa un generador de ceros
        #
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            initial_value=b_init(
                shape=(self.units,),
                dtype="float32",
            ),
            trainable=True,
        )

    def call(self, inputs):
        #
        # Implementa la computación que realiza la capa
        #
        return tf.matmul(inputs, self.w) + self.b


#
# Instanciación de la capa
#
linear_layer = SimpleDense(4)

#
# Este código es equivalente a llamar a `build(input_shape)` y crear los pesos
#
y = linear_layer(tf.ones((2, 2)))


assert len(linear_layer.weights) == 2
assert len(linear_layer.trainable_weights) == 2

In [3]:
#
# Ejemplo 2.
# Uso de `add_weight()` para crear los pesos
#
class SimpleDense(tf.keras.layers.Layer):
    def __init__(self, units=32):
        super(SimpleDense, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,),
            initializer="random_normal",
            trainable=True,
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [4]:
#
# Ejemplo 3.
# Las capas pueden tener pesos no entrenables, los cuales deben ser
# actualizados manualmente durante la llamada a `call()`
#
class ComputeSum(tf.keras.layers.Layer):
    def __init__(self, input_dim):
        super(ComputeSum, self).__init__()
        self.total = tf.Variable(
            initial_value=tf.zeros((input_dim,)),
            trainable=False,
        )

    def call(self, inputs):
        self.total.assign_add(tf.reduce_sum(inputs, axis=0))
        return self.total


my_sum = ComputeSum(2)
x = tf.ones((2, 2))

y = my_sum(x)
print(y.numpy())  # [2. 2.]

y = my_sum(x)
print(y.numpy())  # [4. 4.]

assert my_sum.weights == [my_sum.total]
assert my_sum.non_trainable_weights == [my_sum.total]
assert my_sum.trainable_weights == []

[2. 2.]
[4. 4.]


Propiedad weights
--

In [5]:
#
# Retorna una lista con todas las variables/pesos de una capa
#
linear_layer.weights

[<tf.Variable 'simple_dense/Variable:0' shape=(2, 4) dtype=float32, numpy=
 array([[ 0.03067854,  0.03787955,  0.02552752,  0.06423488],
        [-0.02314473,  0.02136273, -0.00477252,  0.00825742]],
       dtype=float32)>,
 <tf.Variable 'simple_dense/Variable:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]

Propiedad trainable_weights
--

In [6]:
#
# Retorna una lista de todos los pesos entrenables de la capay que son
# actualizados en cada iteración del algoritmo de gradiente descendente
#
linear_layer.trainable_weights

[<tf.Variable 'simple_dense/Variable:0' shape=(2, 4) dtype=float32, numpy=
 array([[ 0.03067854,  0.03787955,  0.02552752,  0.06423488],
        [-0.02314473,  0.02136273, -0.00477252,  0.00825742]],
       dtype=float32)>,
 <tf.Variable 'simple_dense/Variable:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]

Propiedad non_trainable_weights
--

In [7]:
#
# Retorna una lista con los pesos no entrenables
#
linear_layer.non_trainable_weights

[]

Propiedad get_weights
--

In [8]:
#
# Retorna los pesos de la capa como arrays de NumPy
#
layer_a = tf.keras.layers.Dense(
    1,
    kernel_initializer=tf.constant_initializer(1.0),
)
a_out = layer_a(
    tf.convert_to_tensor([[1.0, 2.0, 3.0]]),
)
layer_a.get_weights()

[array([[1.],
        [1.],
        [1.]], dtype=float32),
 array([0.], dtype=float32)]

In [9]:
layer_b = tf.keras.layers.Dense(
    1,
    kernel_initializer=tf.constant_initializer(2.0),
)
b_out = layer_b(
    tf.convert_to_tensor([[10.0, 20.0, 30.0]]),
)
layer_b.get_weights()

[array([[2.],
        [2.],
        [2.]], dtype=float32),
 array([0.], dtype=float32)]

In [10]:
layer_b.set_weights(
    layer_a.get_weights(),
)
layer_b.get_weights()

[array([[1.],
        [1.],
        [1.]], dtype=float32),
 array([0.], dtype=float32)]

Propiedad set_weights
--

Actualiza los pesos de la capa a partir de arrays de NumPy. Ver el ejemplo anterior.

Propiedad get_config
--

In [11]:
#
# Retorna la configuración de la capa
#
layer_a.get_config()

{'name': 'dense',
 'trainable': True,
 'dtype': 'float32',
 'units': 1,
 'activation': 'linear',
 'use_bias': True,
 'kernel_initializer': {'class_name': 'Constant', 'config': {'value': 1.0}},
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'kernel_regularizer': None,
 'bias_regularizer': None,
 'activity_regularizer': None,
 'kernel_constraint': None,
 'bias_constraint': None}

Método add_loss()
--

In [12]:
#
# Adiciona tensores de pérdida potencialmente dependientes de las capas de
# entrada.
#
# Ejemplo 1.
# En este ejemplo el metodo es llamado directamente durante la construcción
# de la capa.
#
class MyLayer(tf.keras.layers.Layer):
    def call(self, inputs):
        self.add_loss(
            tf.abs(
                tf.reduce_mean(inputs),
            ),
        )
        return inputs

In [13]:
#
# Ejemplo 2
# Llamada de add_loss directamente en un modelo funcional.
#
inputs = tf.keras.Input(shape=(10,))
x = tf.keras.layers.Dense(10)(inputs)
outputs = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs, outputs)
model.add_loss(tf.abs(tf.reduce_mean(x)))

In [14]:
#
# Ejemplo 3
# Llamada de add_loss referenciando una variable de una de las capas.
# En este ejemplo funcina como una regularización
#
inputs = tf.keras.Input(shape=(10,))
d = tf.keras.layers.Dense(10)
x = d(inputs)
outputs = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs, outputs)
model.add_loss(lambda: tf.reduce_mean(d.kernel))

Propiedad losses
--

In [15]:
#
# Retorna una lista con las pérdidas adicionadas usando add_loss
#
import numpy as np


class MyLayer(tf.keras.layers.Layer):
    def call(self, inputs):
        self.add_loss(
            tf.abs(tf.reduce_mean(inputs)),
        )
        return inputs


l = MyLayer()
l(np.ones((10, 1)))
l.losses

[<tf.Tensor: shape=(), dtype=float32, numpy=1.0>]

In [16]:
inputs = tf.keras.Input(shape=(10,))
x = tf.keras.layers.Dense(10)(inputs)
outputs = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs, outputs)

len(model.losses)

0

In [17]:
model.add_loss(tf.abs(tf.reduce_mean(x)))
len(model.losses)

1

In [18]:
inputs = tf.keras.Input(shape=(10,))
d = tf.keras.layers.Dense(10, kernel_initializer="ones")
x = d(inputs)
outputs = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs, outputs)
model.add_loss(lambda: tf.reduce_mean(d.kernel))
model.losses

[<tf.Tensor: shape=(), dtype=float32, numpy=1.0>]

Método add_metric()
--

In [19]:
#
# Adiciona una métrica a la capa
#
# Ejemplo 1.
# LLamada dentro de call().
#
class MyMetricLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(MyMetricLayer, self).__init__(name="my_metric_layer")
        self.mean = tf.keras.metrics.Mean(name="metric_1")

    def call(self, inputs):
        self.add_metric(self.mean(inputs))
        self.add_metric(
            tf.reduce_sum(inputs),
            name="metric_2",
        )
        return inputs

In [20]:
#
# Ejemplo 2.
# Llamada dentro de un modelo funcional
#
inputs = tf.keras.Input(shape=(10,))
x = tf.keras.layers.Dense(10)(inputs)
outputs = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs, outputs)
model.add_metric(
    tf.math.reduce_sum(x),
    name="metric_1",
)

Propiedad metric
---

In [21]:
#
# Lista las métricas addicionadas usando add_metric
#
input = tf.keras.layers.Input(shape=(3,))
d = tf.keras.layers.Dense(2)
output = d(input)

d.add_metric(
    tf.reduce_max(output),
    name="max",
)

d.add_metric(
    tf.reduce_min(output),
    name="min",
)

[m.name for m in d.metrics]

['max', 'min']